# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. 

Практическая работа разделена на 2 части: 
1. Построение нграмой языковой модели - основная часть, 10 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 6 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

In [1]:
import numpy as np
from collections import defaultdict
from typing import List, Dict, Tuple
import re

# 1. Построение нграмной языковой модели. (10 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: 
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$

Поскольку униграмы не содержат в себе какого-дибо контекста, вероятность униграмы можно посчитать поделив кол-во этой слова на общее количество слов в обучающей выборке. 


In [2]:
# в первую очередь нам понадобится подсчитать статистику по обучающей выборке 
def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
    ngrams = defaultdict(int)
    # TODO реализуйте функцию, которая подсчитывает все 1-gram 2-gram ... order-gram ngram'ы в тексте
    for _string in train_text:
        if bos:
            _string = '<s> ' + _string
        if eos:
            _string = _string + ' </s>'
        words = _string.split()
        for current_order in (range(1, order+1)):
            for i in range(len(words) - current_order + 1):
                ngram = words[i : i + current_order]
                if tuple(ngram) not in ngrams.keys():
                    ngrams[tuple(ngram)] = 1
                else:
                    ngrams[tuple(ngram)] += 1
    # 
    return dict(ngrams)

In [3]:
def test_count_ngrams():
    assert count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True) == {
        ('<s>',): 1, 
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=True) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=2, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1,
        ('привет', 'привет'): 1,
        ('привет', 'как'): 1,
        ('как', 'дела'): 1
    }    
    assert count_ngrams(['привет ' * 6], order=2, bos=False, eos=False) == {
        ('привет',): 6, 
        ('привет', 'привет'): 5
    }
    result = count_ngrams(['практическое сентября',
                           'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=5)
    assert result[('<s>',)] == 3
    assert result[('32',)] == 3
    assert result[('<s>', 'в', 'офлайне', 'в', '32')] == 1
    assert result[('офлайне', 'в', '32', '12', '</s>')] == 1
    print('Test 1a passed')
    
    
test_count_ngrams()  

Test 1a passed



Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 

Чтобы избежать данного недостатка, вводится специальное сглаживание - add-k сглаживание ([Additive, Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.


In [4]:
# функция подсчета вероятности через количество со сглаживанием Лапласа
def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
    # подсчитывет ngram со сглаживанием Лапласа
    # TODO
    try:
        ngram_count = counts[ngram]
    except KeyError:
        ngram_count = 0
    if len(ngram) > 1:
        condition_count = counts[ngram[:-1]]
    else:
        condition_count = sum(value for key, value in counts.items() if len(key) == 1)
    V = V if V is not None else len([key for key in counts if len(key) == 1])
    prob = (ngram_count + k) / (condition_count + k * V)
    return prob

In [5]:
def test_calculate_ngram_prob():
    counts = count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)
    assert calculate_ngram_prob(('в', 'офлайне'), counts) == 0.5
    assert calculate_ngram_prob(('в', ), counts) == 4/25
    assert calculate_ngram_prob(('в', ), counts, k=0.5) == (4+0.5)/(25+0.5*12)
    assert calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts) == 1.0
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=1) == 0.1875
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=0) == 0.0
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=1) == 0.0625
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

    print("Test 1.b passed")
    

test_calculate_ngram_prob()  

Test 1.b passed


Основной метрикой язковых моделей является перплексия. 

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$


In [6]:
# Языковая модель 
class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = eos
        self.bos = bos
        self.k = k
        self.vocab = predefined_vocab
        self.ngrams_count = None
        
    @property
    def V(self) -> int:
        return len(self.vocab)
    
    def fit(self, train_text: List[str]) -> None:
        # TODO
        # Подсчет vocab и ngrams_count по обучающей выборке
        ngrams = defaultdict(int)
        for _string in train_text:
            if self.bos:
                _string = '<s> ' + _string
            if self.eos:
                _string = _string + ' </s>'
            words = _string.split()
            for current_order in (range(1, self.order+1)):
                for i in range(len(words) - current_order + 1):
                    ngram = words[i : i + current_order]
                    if tuple(ngram) not in ngrams.keys():
                        ngrams[tuple(ngram)] = 1
                    else:
                        ngrams[tuple(ngram)] += 1
        self.ngrams_count = dict(ngrams)
        self.vocab = self.vocab if self.vocab is not None else [key for key in self.ngrams_count if len(key) == 1]
        pass
                
    
    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        # TODO 
        # считаем логарифм вероятности конкретной нграмы
        ngram = tuple(ngram) if isinstance(ngram, list) else ngram
        try:
            ngram_count = self.ngrams_count[ngram]
        except KeyError:
            ngram_count = 0
        if len(ngram) > 1:
            try:
                condition_count = self.ngrams_count[ngram[:-1]]
            except KeyError:
                condition_count = 0
        else:
            condition_count = sum(value for key, value in self.ngrams_count.items() if len(key) == 1)
        prob = (ngram_count + self.k) / (condition_count + self.k * self.V)
        return np.log(prob)
           
    def predict_log_proba(self, words: List[str]) -> float:
        if self.bos:
            words = ['<s>'] + words
        if self.eos:
            words = words + ['</s>']
        logprob = 0
        # TODO 
        # применяем chain rule, чтобы посчитать логарифм вероятности всей строки
        for i in range(1, len(words) + 1):
            ngram = tuple(words[max(0, i - self.order):i])
            logprob += self.predict_ngram_log_proba(ngram)
        return logprob
        
    def ppl(self, text: List[str]) -> float:
        #TODO 
        # подсчет перплексии
        # Для того, чтобы ваш код был численно стабильным, 
        #    не считайте формулу напрямую, а воспользуйтесь переходом к логарифмам вероятностей
        #
        words = []
        for _string in text:
            if self.bos:
                _string = '<s> ' + _string
            if self.eos:
                _string = _string + ' </s>'
            words.extend(_string.split())

        logprob = 0
        for i in range(1, len(words) + 1):
            ngram = tuple(words[max(0, i - self.order):i])
            logprob += self.predict_ngram_log_proba(ngram)
        perplexity = np.exp(-logprob / len(words))
        return perplexity

In [7]:
def test_lm():
    train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
    global lm
    lm = NgramLM(order=2)
    lm.fit(train_data)
    assert lm.V == 14
    assert np.isclose(lm.predict_log_proba(['мы']), lm.predict_log_proba(["если"]))
    assert lm.predict_log_proba(["по-моему"]) > lm.predict_log_proba(["если"]) 
    
    gt = ((3+1)/(41 + 14) * 1/(3+14))**(-1/2)
    ppl = lm.ppl([''])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    gt = ((3+1)/(41 + 14) * 1/(3+14) * 1/(14)) ** (-1/3)
    ppl = lm.ppl(['ЧТО'])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]
    ppl = lm.ppl(test_data)
    assert round(ppl, 2) == 7.33, f"{ppl}"
test_lm()

# 2. Предсказания с помощью языковой модели (6 балла)

In [8]:
def predict_next_word(lm: NgramLM, prefix: List[str], topk=4):
    # TODO реализуйте функцию, которая предсказывает продолжение фразы. 
    # верните topk наиболее вероятных продолжений фразы prefix
    # Предобработка фразы prefix
    prefix_words_list = []
    for _string in prefix:
        prefix_words_list.append(_string.split())
    
    # Считаем вероятности для n-грамм, которые состоят из префикса и каждого слова в словаре модели
    candidates_list = []
    for prefix_words in prefix_words_list:
        candidates = {}
        for word in lm.vocab:
            ngram = prefix_words[-(lm.order - 1):] + [word[0]]
            ngram_logprob = lm.predict_ngram_log_proba(ngram)
            candidates[word] = ngram_logprob
        candidates_list.append(candidates)

    # Сортируем полученные вероятности, возвращаем topk слов с наиболее высокими вероятностями
    candidates_sorted_list = []
    for candidates in candidates_list:
        sorted_candidates = sorted(candidates.items(), key=lambda x: x[1], reverse=True)
        sorted_words = [word[0] for word, _ in sorted_candidates[:topk]]
        candidates_sorted_list.append(sorted_words)
    #return word
    return candidates_sorted_list
    

In [9]:
prefix = ["вот бы сейчас сойти с ума"]
predict_next_word(lm, prefix)

[['прямо', 'по-моему', '<s>', 'мы']]

Попробуйте обучить ngram языковую модель на нескольких стихотворениях. Не забудьте трансформировать стихотворение в удобный для ngram модели формат (как сделать так, чтобы модель моделировала рифму?). 
Попробуйте сгенерировать продолжение для стихотворения с помощью такой языковой модели. 

In [10]:
#Your code here
def preprocess_text(text: str) -> List[str]:
    text = text.lower()
    return " ".join(re.findall(r"\w+\b", text))

def get_train_data_for_poems(file_path: str) -> List[str]:
    with open(file_path, 'r') as file:
        data = file.read().splitlines()
    train_data = []
    for line in data:
        if line:
            train_data.append(preprocess_text(line))
    return train_data

In [11]:
# Возможно, удобный для ngram модели формат – список из строк нескольких стихотворений (?)
# Можно попробовать взять стихи только одного стихотворного размера, например, ямба
train_data = get_train_data_for_poems("poems.txt")
lm = NgramLM(order=2)
lm.fit(train_data)
prefix = [preprocess_text("Испытанный, стою на грани. Земных свершений жизни жду. Они взметнутся в урагане, В экстазе, в страсти и в"),
          preprocess_text("Но возрожденья весть живая Уж есть в пролётных журавлях, И, их глазами провожая, Стоит красавица степная С румянцем сизым на")]
predict_next_word(lm, prefix)

[['ней', 'лесу', 'небе', 'ночи'], ['мир', 'омертвелых', 'тыщу', 'каменные']]